<a href="https://colab.research.google.com/github/ykitaguchi77/AutoGen/blob/main/Autogen%E3%82%B5%E3%83%B3%E3%83%97%E3%83%AB_%E3%82%B0%E3%83%AB%E3%83%BC%E3%83%97%E3%83%81%E3%83%A3%E3%83%83%E3%83%88%EF%BC%88%E3%83%A1%E3%83%8B%E3%83%A5%E3%83%BC%E8%80%83%E6%A1%88%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autogenについて
Microsoft Researchから登場したマルチエージェントによるLLMアプリケーションを実現するフレームワーク。役割を設定したエージェントを定義することで、エージェントがお互いに相談しながらタスクを実行できる。インターネットアクセスやコードの実行、ファンクションなども活用しながら高度な操作ができる。

- 公式サイト  
  https://microsoft.github.io/autogen/  
- Github  
  https://github.com/microsoft/autogen  
- ブログ  
  AutoGen: Enabling next-generation large language model applications  
  https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/  
- arxivペーパー  
  AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation  
  https://arxiv.org/abs/2308.08155  

## 注意
- 実行には、OpenAIのAPIキーが必要。
- 複雑な処理や高度な連携にはGPT-4のアクセスが必要。
- コードの修正などが繰り返されるケースではAPIアクセスが多発しコストが嵩むので注意。
- タスクによっては、エージェント間でコードやソースなどを共有する必要があり、大量の入力が発生するため、16Kや32Kに対応したモデルでないと動作しない（Rate Limitエラーにひっかかる）。
- Web情報の取得やデータ分析などは16K、32Kモデル必須。


### STEP 1：準備
pipでインストール。  
ランタイム環境にインストールされてない場合は以下実行。

In [18]:
%%capture --no-stderr
%pip install pyautogen~=0.1.0

### STEP2：設定
設定はOAI_CONFIGファイルから読み込むこともできるが、今回は直接設定する。  
"api_key"にOpenAIのAPIキーを登録しておく。  
Azureを利用する場合は本家のサンプルを参照のこと。  

In [19]:
# APIキー設定スクリプト
# 目的：Google Driveからapi.txtファイルを読み込み、必要なAPIキーを環境変数として設定する
# 使用するAPI：OpenAI API, SerpAPI, Google Custom Search Engine API

# APIの設定
from google.colab import drive
drive.mount("/content/drive")

with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

# APIキーの準備
# #ngrok_aceess_token = key[5]
#openai_api_key = key[3]
# deepl_auth_key = key[1]
# serp_api_key = key[7]

import os
os.environ["OPENAI_API_KEY"] = key[3]
os.environ["SERPAPI_API_KEY"] = key[7]
os.environ["GOOGLE_CSE_ID"] = key[9]
os.environ["GOOGLE_API_KEY"] = key[11]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import autogen

config_list = [
    {
        "model": "gpt-4o",
        "api_key": key[3],
        "api_type": "open_ai",
        "api_base": "https://api.openai.com/v1",
        "api_version": None,
#        "request_timeout": 120,
#        "max_retry_period": 90,
#        "retry_wait_time": 5,
    },
    {
        "model": "gpt-4o-mini",
        "api_key": key[3],
        "api_type": "open_ai",
        "api_base": "https://api.openai.com/v1",
        "api_version": None,
    },
]

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


### エージェントの定義
今回の例では、以下の４つのエージェントを定義し、1週間分の夕食の献立を相談しながら決めてもらう。  
- floor_manager  
ユーザーとの対話を担当  
- chef  
メニューを考案する  
- doctor  
医学的立場からメニューを修正する  
- kitchen_manager  
メニューに必要な食材などを調達する
  
「system_message=」でエージェントの役割を定義する。  
  
今回はコードの生成と実行はしないが、必要な場合は  
「code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},」  
を利用する。  
  
「groupchat = autogen.GroupChat(agents=[floor_manager, chef, doctor, kitchen_manager], messages=[], max_round=12)」  
で、参加させるエージェントと実行ラウンド数を決める。  
  
人間の介入を受け付ける場合は、以下を変更
「human_input_mode="TERMINATE"」→「human_input_mode="ALWAYS"」

In [14]:
llm_config = {"config_list": config_list}
floor_manager = autogen.UserProxyAgent(
   name="floor_manager",
   system_message="ChefやDoctor、Kitchen_Managerと相談しながら調理や食材に関する課題を解決してください。満足が得られなければ再度フィードバックをかけてください",
   code_execution_config={"last_n_messages": 3, "work_dir": "groupchat"},
   human_input_mode="TERMINATE"
)
chef = autogen.AssistantAgent(
    name="chef",
    system_message="世界中の料理を知り尽くした料理人です。健康面はdoctorが検討するので、考慮する必要はありません。味についてのみ検討したレシピを考案し、最高の料理を提供します。考案したメニューをdoctorと相談して、健康面を考慮しながら修正してください。メニューが決定したらfloor_managerに食材リストの提出を依頼してください。",
    llm_config=llm_config,
)
doctor = autogen.AssistantAgent(
    name="doctor",
    system_message="chefが提案したメニューを医学的な立場で検証し、floor_managerに修正を依頼してください。",
    llm_config=llm_config,
)
kitchen_manager = autogen.AssistantAgent(
    name="kitchen_manager",
    system_message="料理にかかる費用や必要な食材や調味料を管理します。chefが考案しメニューから必要な食材を検討し、floor_managerに知らせてください。",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[floor_manager, chef, doctor, kitchen_manager], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

### チャット開始
議論してもらうテーマを与えると議論が開始される。  
  
実行時に「chef(to chat_manager):」のように誰宛ての会話なのかが表示される点に注目。

In [16]:
floor_manager.initiate_chat(
    manager, message=
    "私は高血圧の持病を持っていますが、食に熱心で、常においしいものを食べたいと考えています。このことを考慮して、明日の夕食の献立を考案し、それに必要な食材を調達するためのリストを生成してください。"
)
# type exit to terminate the chat

floor_manager (to chat_manager):

私は高血圧の持病を持っていますが、食に熱心で、常においしいものを食べたいと考えています。このことを考慮して、明日の夕食の献立を考案し、それに必要な食材を調達するためのリストを生成してください。

--------------------------------------------------------------------------------
chef (to chat_manager):

かしこまりました。高血圧を考慮せず、まずは味を追求した明日の夕食メニューを考案します。その後、doctorと相談して健康面を考慮した修正を行います。

### 夕食メニュー（味を優先）

#### メインディッシュ: クリーミーなカルボナーラ
- 生クリームとパルメザンチーズで仕上げた濃厚カルボナーラスパゲッティ
- サクッとしたパンチェッタのトッピング
- フレッシュパセリ

#### サイドディッシュ: ガーリックブレッド
- バターたっぷりの自家製ガーリックブレッド

#### サラダ: シーザーサラダ
- ロメインレタス
- クルトン
- パルメザンチーズ
- シーザードレッシング

### 次のステップ

1. 医師と相談し、健康面からの修正を受けた新しいメニューを提案します。
2. 修正メニューに基づいた食材リストを生成し、floor_managerに調達を依頼します。

次は、doctorに健康面の修正をお願いしましょう。

--------------------------------------------------------------------------------
doctor (to chat_manager):

高血圧に配慮し、美味しさを保ちながら健康的なメニューに修正する提案をします。脂肪や塩分を控えめにし、野菜の摂取量を増やす方針で改善していきましょう。

### 医学的観点からの修正提案

#### メインディッシュ: クリーミーなカルボナーラ（修正後）
- 生クリームを使用せず、代わりに低脂肪ミルクと無塩のチキンストックを使用
- パルメザンチーズの量を控えめに
- パンチェッタの代わりに薄切りのターキーや鶏肉の使用を推奨
- 全粒粉スパゲッティを選択

#**公式のサンプル**

###シンプルな会話

In [20]:
import os
from autogen import ConversableAgent

agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ.get("OPENAI_API_KEY")}]},
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

In [21]:
reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "user"}])
print(reply)

Sure, here is a light-hearted one for you:

Why don't scientists trust atoms?

Because they make up everything!


###Agentを増やす

In [34]:
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4o", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4o", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

In [ ]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.",
                           max_turns=10,
                           max_consecutive_auto_reply=1,)

### Good Byeが含まれていたら会話を終了

In [31]:
joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
    is_termination_msg=lambda msg: "good bye" in msg["content"].lower(),
)

result = joe.initiate_chat(cathy, message="Cathy, tell me a joke and then say the words GOOD BYE.")

joe (to cathy):

Cathy, tell me a joke and then say the words GOOD BYE.

--------------------------------------------------------------------------------
cathy (to joe):

Why don't scientists trust atoms? Because they make up everything!

GOOD BYE!

--------------------------------------------------------------------------------


###数当てゲーム実装

In [ ]:
import os
from autogen import ConversableAgent

agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. You have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="NEVER",  # never ask for human input
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

###ヒトも会話に参加する

In [40]:
import os
from autogen import ConversableAgent

agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. You have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="NEVER",  # never ask for human input
)

human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
)

# Start a chat with the agent with number with an initial guess.
result = agent_with_number.initiate_chat(
    human_proxy,  # this is the same agent with the number as before
    message="I have a number between 1 and 100. Guess it!",
)

agent_with_number (to human_proxy):

I have a number between 1 and 100. Guess it!

--------------------------------------------------------------------------------
Provide feedback to agent_with_number. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 43
human_proxy (to agent_with_number):

43

--------------------------------------------------------------------------------
agent_with_number (to human_proxy):

Too low.

--------------------------------------------------------------------------------
Provide feedback to agent_with_number. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 52
human_proxy (to agent_with_number):

52

--------------------------------------------------------------------------------
agent_with_number (to human_proxy):

Too low.

--------------------------------------------------------------------------------
Provide feedback to agent_with_number. Press enter to skip and use auto-reply, or type 'exit

###中止するかどうかをその都度聞く

In [ ]:
agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. "
    "In the first game, you have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    max_consecutive_auto_reply=1,  # maximum number of consecutive auto-replies before asking for human input
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="TERMINATE",  # ask for human input until the game is terminated
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

#**Autogen Sample Scripts**

Multiagents

https://github.com/Poly186-AI-DAO/AutoGen-Example-Scripts/tree/master



### Multiagent developent of the python code

In [ ]:

from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, GroupChat, GroupChatManager


gpt4o_config = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0,
    "config_list":  [{"model": "gpt-4o", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}],
    "request_timeout": 120,
}
user_proxy = UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
)
engineer = AssistantAgent(
    name="Engineer",
    llm_config=gpt4o_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
Speak Japanese
''',
)
scientist = AssistantAgent(
    name="Scientist",
    llm_config=gpt4o_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code. Speak Japanese"""
)
planner = AssistantAgent(
    name="Planner",
    system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
Plans should be in Japanese.
''',
    llm_config=gpt4o_config,
)
executor = UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "paper"},
)
critic = AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gpt4o_config,
)
groupchat = GroupChat(agents=[user_proxy, engineer, scientist, planner, executor, critic], messages=[], max_round=50)
manager = GroupChatManager(groupchat=groupchat, llm_config=gpt4o_config)
user_proxy.initiate_chat(
    manager,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
""",
)

In [59]:
### マルチエージェントの成果物

!pip install arxiv scikit-learn

import arxiv
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

# 現在の日付と1週間前の日付を取得
now = datetime.datetime.now()
last_week = now - datetime.timedelta(weeks=1)

# 日付範囲の文字列を作成
date_query = f"submittedDate:[{last_week.strftime('%Y%m%d')} TO {now.strftime('%Y%m%d')}]"

# ArXiv APIを使用してLLM関連の論文を検索
search_query = f'("large language model" OR LLM) AND {date_query}'
search = arxiv.Search(
    query=search_query,
    max_results=20,
    sort_by=arxiv.SortCriterion.SubmittedDate,
    sort_order=arxiv.SortOrder.Descending,
)

# 論文をリスト化
papers = []
abstracts = []
try:
    for result in search.results():
        paper = {
            "title": result.title,
            "authors": ", ".join(author.name for author in result.authors),
            "published": result.published,
            "url": result.entry_id,
            "abstract": result.summary
        }
        papers.append(paper)
        abstracts.append(result.summary)
except Exception as e:
    print(f"エラーが発生しました: {e}")

# TF-IDFベクトル化
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(abstracts)

# クラスタ数の最適化
best_n_clusters = 2
best_score = -1
for n_clusters in range(2, min(10, len(abstracts))):  # 最大10クラスタに限定
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(X)
    score = silhouette_score(X, labels)
    if score > best_score:
        best_score = score
        best_n_clusters = n_clusters

# 最適なクラスタ数でKMeans実行
kmeans = KMeans(n_clusters=best_n_clusters, random_state=42)
labels = kmeans.fit_predict(X)

# 各クラスタに対応するドメイン名を決定するためのマッピング
cluster_to_domain = {
    0: "AI and ML",
    1: "NLP",
    2: "Computer Vision",
    3: "Ethics and Safety",
    4: "Applications",
    # 必要に応じて拡張
}

# Markdown形式のテーブルを作成
markdown_table = "| 論文タイトル | 著者 | 発行日 | URL | ドメイン |\n"
markdown_table += "|-------------|------|--------|-----|----------|\n"

# 結果の表示とテーブルの作成
for i, paper in enumerate(papers):
    domain = cluster_to_domain.get(labels[i], "未分類")
    print(f"Title: {paper['title'][:30]}..., Domain: {domain}")
    markdown_table += f"| {paper['title']} | {paper['authors']} | {paper['published'].date()} | [Link]({paper['url']}) | {domain} |\n"

# 最終的なテーブルを表示
print("\n最終的なMarkdownテーブル:\n")
print(markdown_table)

<ipython-input-59-7bfbbefa6890>:30: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Title: Towards Trustworthy LLMs for C..., Domain: Applications
Title: Unraveling and Mitigating Safe..., Domain: NLP
Title: MiRAGeNews: Multimodal Realist..., Domain: Computer Vision
Title: AttnGCG: Enhancing Jailbreakin..., Domain: NLP
Title: Semi-Supervised Learning of No..., Domain: Applications
Title: SimpleStrat: Diversifying Lang..., Domain: 未分類
Title: Mentor-KD: Making Small Langua..., Domain: 未分類
Title: PEAR: A Robust and Flexible Au..., Domain: AI and ML
Title: AgentHarm: A Benchmark for Mea..., Domain: NLP
Title: The Impact of Visual Informati..., Domain: AI and ML
Title: Software Engineering and Found..., Domain: AI and ML
Title: SuperCorrect: Supervising and ..., Domain: 未分類
Title: From Interaction to Impact: To..., Domain: Computer Vision
Title: Hypothesis-only Biases in Larg..., Domain: Computer Vision
Title: The structure of the token spa..., Domain: Ethics and Safety
Title: Science is Exploration: Comput..., Domain: AI and ML
Title: SubZero: Random Subspace Zerot..., Do

In [61]:
# Import necessary libraries
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

llm_config = {
    "request_timeout": 600,
    "seed": 42,
    "config_list":  [{"model": "gpt-4o", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}],
    "temperature": 0,
}

# Construct agents
assistant = AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "web"},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

# Start a conversation
user_proxy.initiate_chat(
    assistant,
    message="""
Tell me about this project, and the libary, then also tell me what I can use it for: https://www.amed.go.jp/about-navi.html
""",
)

user_proxy (to assistant):


Tell me about this project, and the libary, then also tell me what I can use it for: https://www.amed.go.jp/about-navi.html


--------------------------------------------------------------------------------
assistant (to user_proxy):

To provide you with detailed information about the project and the library mentioned in the link, I will first gather the information available on the web page. Then, I will explain what the project and library are about and how you can use them. 

Let me retrieve the content of the webpage first. 

```python
import requests
from bs4 import BeautifulSoup

url = "https://www.amed.go.jp/about-navi.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Print the content of the page
print(soup.get_text())
```

Please execute the code above to retrieve the content of the webpage. Once you provide the content, I will be able to proceed with explaining the project and how you can use it.

--------

user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 




AMEDを知りたい方 | 国立研究開発法人日本医療研究開発機構


























English Site




サイト内検索









採用情報
アクセス
お問い合わせ


メルマガ






AMEDについて
開閉ボタン





AMEDについて



理事長挨拶
機構概要
理念、運営方針等
中長期計画・年度計画・報告等
海外事業所の活動報告
国内外機関との連携







事業紹介
開閉ボタン





事業紹介


分野別紹介

医薬品プロジェクト
医療機器・ヘルスケアプロジェクト
再生・細胞医療・遺伝子治療プロジェクト
ゲノム・データ基盤プロジェクト
疾患基礎研究プロジェクト
シーズ開発・研究基盤プロジェクト
CiCLE
健康・医療分野におけるムーンショット型研究開発等事業
ワクチン開発・生産体制強化戦略関連事業（SCARDA関連他）
第一期中長期計画期間におけるプロジェクト

事業部紹介

創薬事業部
医療機器・ヘルスケア事業部
再生・細胞医療・遺伝子治療事業部
ゲノム・データ基盤事業部
疾患基礎研究事業部
シーズ開発・研究基盤事業部
革新基盤創成事業部
研究開発統括推進室
実用化推進部
国際戦略推進部
先進的研究開発戦略センター（SCARDA）







公募情報
開閉ボタン





公募情報


公募情報検索

公募情報検索
公募相談窓口

公募情報一覧

公募予告一覧
公募情報一覧
採択情報一覧

カテゴリ別一覧

掲載日順一覧
分野別一覧
部課室別一覧

事務処理説明書・様式集、研究開発実施上の留意事項

事務処理説明書・様式集（委託）
事務処理説明書・様式集（補助）
研究費の運用
データの取扱いに関する基本方針
データシェアリングポリシー
データの第三者提供と利活用を円滑・適正に進めるための文書
医薬品開発のチェック項目
医療機器のチェックポイント
再生医療研究事業の進捗管理
知的財産
研究公正







事業の成果
開閉ボタン





事業の成果



プレスリリース
成果情報
研究成果報告書
研究事業成果集
受賞一覧
先駆